<a href="https://colab.research.google.com/github/nishitgala89/ProjectDissertation/blob/main/Code/2_DataPreparation_RemovingDuplicates/Morningstar_new_Dataframe_RemoveDuplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
#morningstar_df = pd.read_csv("/content/sample_data/Asia_Developed.csv")


In [34]:
morningstar_data_url = "https://raw.githubusercontent.com/nishitgala89/ProjectDissertation/main/Datasheets/1_Preparation/1_All_Equities_MorningStar.csv"
morningstar_df = pd.read_csv(morningstar_data_url).set_index("SecId")
morningstar_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2761 entries, F0GBR04AX4 to F000010KD8
Data columns (total 49 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Name                              2761 non-null   object 
 1   PriceCurrency                     2761 non-null   object 
 2   TenforeId                         2753 non-null   object 
 3   LegalName                         2761 non-null   object 
 4   ClosePrice                        2761 non-null   float64
 5   StarRatingM255                    2761 non-null   int64  
 6   SustainabilityRank                2761 non-null   int64  
 7   QuantitativeRating                1047 non-null   float64
 8   CategoryName                      2761 non-null   object 
 9   Yield_M12                         2756 non-null   float64
 10  GBRReturnD1                       2761 non-null   float64
 11  GBRReturnW1                       2761 non-null   float64
 

## Scrapped Dataframe

### 2) Uploading the UK Scrapped Data

In [ ]:
uk_scrapped_url = "https://raw.githubusercontent.com/nishitgala89/ProjectDissertation/main/Datasheets/1_Preparation/2_UK_scrapped_df.csv"
uk_scrapped_df = pd.read_csv(uk_scrapped_url).set_index("SecId")

### 3) Uploading the Europe Developed Scrapped Data

In [ ]:
eu_scrapped_url = "https://raw.githubusercontent.com/nishitgala89/ProjectDissertation/main/Datasheets/1_Preparation/2_Europe_scrapped_df.csv"
eu_scrapped_df = pd.read_csv(eu_scrapped_url).set_index("SecId")

### 4) Combining the Scrapped data into a Single Data Frame

In [ ]:
combined_scrapped_df = pd.concat([uk_scrapped_df,eu_scrapped_df],axis=0)
combined_scrapped_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2760 entries, F0GBR04AX4 to F00000LL2Y
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   P_E                               2759 non-null   float64
 1   P_B                               2760 non-null   float64
 2   P_Sales                           2760 non-null   float64
 3   P_CashFlow                        2760 non-null   float64
 4   Dividend_Yield_Percent            2760 non-null   float64
 5   Long_term_earning_Percent         2760 non-null   float64
 6   Historical_earning_Percent        2760 non-null   float64
 7   Sales_growth_percent              2760 non-null   float64
 8   Cashflow_growth_percent           2760 non-null   float64
 9   Book_value_growth_percent         2760 non-null   float64
 10  Equity_Holding                    2760 non-null   object 
 11  Other_Holding                     2760 non-null   int64  
 

Note - There are 2761 records in Morningstar DF and 2760 records in scrapped DF, so need to remove the missing index and bring the both dataframes with same SecId's

In [ ]:
all_morningstar_index = morningstar_df.index.to_list()
len(all_morningstar_index)

2761

In [ ]:
all_scrapped_index = combined_scrapped_df.index.to_list()
len(all_scrapped_index)

2760

In [ ]:
for index in all_scrapped_index:
  try:
    all_morningstar_index.remove(index)
  except:
    print("Error for --> ", index)
all_morningstar_index

['F00000X3G0']

In [ ]:
morningstar_df.drop(index=["F00000X3G0"], inplace=True)

In [ ]:
morningstar_df.reset_index(inplace=True)

In [ ]:
df = morningstar_df.groupby(['CategoryName','ManagerTenure','AverageMarketCapital','StarRatingM255'])['SecId'].count().to_frame()

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184 entries, 0 to 1183
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CategoryName          1184 non-null   object 
 1   ManagerTenure         1184 non-null   float64
 2   AverageMarketCapital  1184 non-null   float64
 3   StarRatingM255        1184 non-null   int64  
 4   SecId                 1184 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 46.4+ KB


In [ ]:
morningstar_df.set_index('SecId', inplace=True)

In [ ]:
(morningstar_df[(morningstar_df.ManagerTenure == 1.5) &  (morningstar_df.StarRatingM255 == 5) 
                  & (morningstar_df.CategoryName == 'Europe Equity Income')
                  & (morningstar_df.AverageMarketCapital == 24636.42657)].index)[0]

'F0000118JH'

In [ ]:
new_index = []
for index,row in df.iterrows():  
  new_index.append((morningstar_df[(morningstar_df.ManagerTenure == row['ManagerTenure']) 
                  &  (morningstar_df.StarRatingM255 == row['StarRatingM255']) 
                  & (morningstar_df.CategoryName == row['CategoryName'] )
                  & (morningstar_df.AverageMarketCapital == row['AverageMarketCapital'])].index)[0])

len(new_index)

1184

In [ ]:
len(set(new_index))

1184

In [ ]:
new_df = pd.DataFrame(data=morningstar_df,index=new_index)

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1184 entries, F0GBR04N5Q to F0GBR04OC1
Data columns (total 49 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Name                              1184 non-null   object 
 1   PriceCurrency                     1184 non-null   object 
 2   TenforeId                         1182 non-null   object 
 3   LegalName                         1184 non-null   object 
 4   ClosePrice                        1184 non-null   float64
 5   StarRatingM255                    1184 non-null   int64  
 6   SustainabilityRank                1184 non-null   int64  
 7   QuantitativeRating                487 non-null    float64
 8   CategoryName                      1184 non-null   object 
 9   Yield_M12                         1180 non-null   float64
 10  GBRReturnD1                       1184 non-null   float64
 11  GBRReturnW1                       1184 non-null   float64
 

In [ ]:
new_scrapped_df = pd.DataFrame(data=combined_scrapped_df,index=new_index)

In [ ]:
new_scrapped_df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1184 entries, F0GBR04N5Q to F0GBR04OC1
Columns: 44 entries, P_E to Size_5_Max
dtypes: float64(40), int64(1), object(3)
memory usage: 416.2+ KB


In [ ]:
new_df.groupby(['CategoryName','ManagerTenure','AverageMarketCapital','StarRatingM255'])['Name'].count().to_frame()

Name
CategoryName         ManagerTenure AverageMarketCapital StarRatingM255      
Denmark Equity       10.92         15684.04591          2                  1
Europe Equity Income 0.75          54955.21402          5                  1
                     1.50          24636.42657          4                  1
                                                        5                  1
                     2.00          50389.39812          2                  1
...                                                                      ...
UK Small-Cap Equity  20.75         1117.73022           3                  1
                     24.42         1174.39149           3                  1
                                                        4                  1
                     30.25         535.79158            1                  1
                     30.67         723.10137            1                  1

[1184 rows x 1 columns]

In [ ]:
new_df.to_csv('3_Morningstar_new_df.csv')
new_scrapped_df.to_csv('3_Scrapped_new_df.csv')